In [ ]:
import os
import shutil
import glob
import pandas as pd
from joblib import Parallel, delayed
from nipype import Node, Workflow
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.fsl.maths import Threshold, UnaryMaths
from nipype.interfaces.ants import ApplyTransforms

In [ ]:
bids_root = os.environ.get('BIDS_ROOT', '/path/to/NeuroMET')

## Rename MRS Mask files 

In [ ]:
mrs_der1 = os.path.join(bids_root, 'derivatives', 'mrs_masks')
mrs_der2 = os.path.join(bids_root, 'derivatives', 'mrs_masks_onlysids')

In [ ]:
# modify sidses and copy masks to mrs_masks_onlysids
src_masks = glob.glob(os.path.join(mrs_der1, 'sub-*/ses-*/anat/*desc-pcc-voxel_mask.nii.gz'))
src_masks.sort()

len(src_masks)

In [ ]:
def copy_masks_to_onlysids(src_mask):
    dest_mask = src_mask.replace(mrs_der1, mrs_der2).replace('_ses-','').replace('/ses-','')
    os.makedirs(os.path.dirname(dest_mask), exist_ok=True)
    shutil.copy(src_mask, dest_mask)
    return dest_mask

Parallel(n_jobs=32)(delayed(copy_masks_to_onlysids)(src_mask) for src_mask in src_masks)

# Coregistration coreg

In [ ]:
os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'
# os.environ['ANTSPATH'] can be set externally if needed
os.environ['FREESURFER_HOME'] = os.environ.get('FREESURFER_HOME', '/path/to/freesurfer')

In [ ]:
nm_dir = os.environ.get('BIDS_ROOT', '/path/to/NeuroMET')
mrs_voxels_dir = os.path.join(nm_dir, 'derivatives', 'mrs_masks_onlysids')

In [ ]:
mrs_masks = glob.glob(os.path.join(mrs_voxels_dir, 'sub-*', 'anat', '*desc-pcc-voxel_mask.nii.gz'))
len(mrs_masks)

In [ ]:
mrs_voxels_dir_onlysids = os.path.join(nm_dir, 'derivatives', 'mrs_masks_onlysids')


In [ ]:
sids = os.listdir(mrs_voxels_dir_onlysids)
sids[:2], len(sids)

In [ ]:
# sids = [sids[0]]
# sids
sids = [
    "sub-NeuroMET00401",
    "sub-NeuroMET04501",
    "sub-NeuroMET04801",
    "sub-NeuroMET04901",
    "sub-NeuroMET06101",
    "sub-NeuroMET07301",
    "sub-NeuroMET08001",
    "sub-NeuroMET08701"
]

In [ ]:
ids_source = Node(IdentityInterface(fields=['sid']), name="ids_source")
ids_source.iterables = [("sid", sids)]
ids_source.synchronize=True

In [ ]:
dg = Node(DataGrabber(infields=['sid'], outfields=['mrs_mask', 't1w', 'transform']), 'datagrabber')
dg.inputs.base_directory = nm_dir
dg.inputs.template = '%s/%s/anat/%s_%s'
dg.inputs.template_args = dict({'mrs_mask': [['derivatives/mrs_masks_onlysids',
                                              'sid',
                                              'sid', 
                                              'desc-pcc-voxel_mask.nii.gz']],
                                't1w': [['derivatives/fmriprep',
                                         'sid',
                                         'sid', 
                                         'acq-UNIbrainDENskull_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz']],
                                'transform': [['derivatives/fmriprep',
                                         'sid',
                                         'sid', 
                                         'acq-UNIbrainDENskull_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5']]})
dg.inputs.sort_filelist = False

In [ ]:
# Count src files 
mrs_masks = glob.glob(os.path.join(nm_dir, 'derivatives', 'mrs_masks_onlysids', 'sub-*', 'anat', '*desc-pcc-voxel_mask.nii.gz'))
t1s = glob.glob(os.path.join(nm_dir, 'derivatives', 'fmriprep', 'sub-*', 'anat', '*acq-UNIbrainDENskull_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'))
transforms = glob.glob(os.path.join(nm_dir, 'derivatives', 'fmriprep', 'sub-*', 'anat', '*acq-UNIbrainDENskull_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'))
print(f"Number of MRS masks: {len(mrs_masks)}")
print(f"Number of T1w images: {len(t1s)}")
print(f"Number of transforms: {len(transforms)}")


In [ ]:
# see sids in common
sids_mrs_masks = [os.path.basename(mrs_mask).split('_')[0] for mrs_mask in mrs_masks]
sids_t1s = [os.path.basename(t1).split('_')[0] for t1 in t1s]
sids_transforms = [os.path.basename(transform).split('_')[0] for transform in transforms]
common_sids = set(sids_mrs_masks) & set(sids_t1s) & set(sids_transforms)
print(f"Common SIDs: {len(common_sids)}")

In [ ]:
from nipype.interfaces.ants import ApplyTransforms
apply_transform = Node(ApplyTransforms(
    dimension=3,
    interpolation="NearestNeighbor"
), name="apply_transform")

In [ ]:
# threshold the mask 
threshold_mask = Node(Threshold(
    thresh=0.5,
    args='-bin',
), name='threshold_mask')

# binarize the mask
binarize_mask = Node(UnaryMaths(operation='bin'), name='binarize_mask')



In [ ]:
# create sink
output_dir = os.path.join(nm_dir, 'derivatives', 'coreg_mrs_masks')
sink = Node(DataSink(base_directory=output_dir), name='sink')

# actual path: _sid_sub-NeuroMET12601/sub-NeuroMET12601_desc-mrs-pcc_mask_trans_thresh_bin.nii.gz
# modify to achieve the path: sub-NeuroMET12601/sub-NeuroMET12601-space-MNI152NLin2009cAsym_desc-mrs-pcc_mask.nii.gz

sink.inputs.regexp_substitutions = [
    (r'_sid_(sub-[^/]+)/', r'\1/anat/'),
    ('_desc-pcc-voxel_mask', '_space-MNI152NLin2009cAsym_desc-pcc-voxel_mask'),
    ('_trans_thresh_bin', '')
]

In [ ]:
tmp_dir = os.environ.get('TMPDIR', os.path.join(os.getcwd(), 'tmp', 'coreg_mrs_voxel_masks'))
wf = Workflow(name='coreg_mrs_voxel_masks')
wf.base_dir = tmp_dir
wf.connect(ids_source,'sid', dg, 'sid')
wf.connect(dg, 'mrs_mask', apply_transform, 'input_image')
wf.connect(dg, 't1w', apply_transform, 'reference_image')
wf.connect(dg, 'transform', apply_transform, 'transforms')
wf.connect(apply_transform, 'output_image', threshold_mask, 'in_file')
wf.connect(threshold_mask, 'out_file', binarize_mask, 'in_file')
wf.connect(binarize_mask, 'out_file', sink, '@coreg_mask')

In [ ]:
sids[0], len(sids)

In [ ]:
wf.run('MultiProc', plugin_args={'n_procs': 32, 'memory_gb': 32, 'raise_insufficient': False})

In [ ]:
coreg_masks = glob.glob(os.path.join(nm_dir, 'derivatives', 'coreg_mrs_masks', '*', 'anat', '*_space-MNI152NLin2009cAsym_desc-*_mask.nii.gz'))

In [ ]:
len(coreg_masks)

In [ ]:
masks_batch = []
for mask in coreg_masks:
    #print(mask.split('/')[-3])
    masks_batch.append(mask.split('/')[-3])

In [ ]:
print("masks_batch=(" + " ".join(masks_batch) + ")")